### Import stuff

In [ ]:
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
# import stuff for tensorboard
from torch.utils.tensorboard import SummaryWriter

### Custom Dataloader

In [22]:
# make a dictionary of possible values to normalize to
val_arr = [chr(32), chr(35), chr(37), chr(42), chr(43), chr(45), chr(46), chr(58), chr(61), chr(64)]
lin = np.linspace(-1, 1, num=10)

norm_dic = {}
for k, lin in zip(val_arr, lin):
    norm_dic[k] = lin
norm_dic

def to_number(x, normalize=True):
    if normalize:
        if (not type(x) is str and (x==None or np.isnan(x))):
            return -1.;
        return norm_dic[x]
        
    else:
        if (not type(x) is str and (x==None or np.isnan(x))):
            return 32;
        return ord(x)

class AsciiToDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, height=40, width=80):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.height = height
        self.width = width
        
    def __len__(self):
        return len(self.img_labels)


    def __getitem__(self, idx, normalize=True):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])

        # create empty array
        if normalize:
            arr = np.zeros([40,80], dtype="float32")
        else:
            arr = np.zeros([40,80], dtype="int")

        # indexes
        i, j = 0, 0

        # iterate through txt file and convert
        with open(img_path) as f:
            for line in f.readlines():
                for ch in line: 
                    if not (ch == "\n"):# leave out newline char
                        arr[j][i] = to_number(ch, normalize)
                    i += 1
                j += 1
                i = 0
        
        label = self.img_labels.iloc[idx, 1]
        return arr, label

### Make datasets and -loaders

In [23]:
annotation_file = "dataset/ascii.csv"
img_dir = "dataset/w80_h40_ascii"

test_dataset = AsciiToDataset(annotation_file, img_dir)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True) #shuffle=false to debug

train_features, train_labels = next(iter(test_dataloader))

### Define Convnet

In [24]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO conv1 is not correct: gets batch size first i think
        self.conv1 = nn.Conv2d(64, 4, 5) # batch size =? input size, output size, kernel size
        self.pool = nn.MaxPool2d(2, 2)  # kernen size, stride
        self.conv2 = nn.Conv2d(4, 64, 5)
        self.fc1 = nn.Linear(119, 120)  # Wrong:
                                            #12*7*12 is the dimension of input at this point:
                                            # 7 = ((((40-4) /2) -4) /2) bc conv has kernel size 5
                                            # and therefore shaves off 2 left and right and maxpool
                                            # halves the size
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        #print(f"before: {x.shape}")
        x = self.pool(F.relu(self.conv1(x)))
        #print(f"after conv1, relu and pool: {x.shape}")
        x = self.pool(F.relu(self.conv2(x)))
        #print(f"after conv2, relu and pool: {x.shape}")
        x = torch.flatten(x, 1)  # flatten to 1D
        #print(f"after flattenin: {x.shape}")
        x = F.relu(self.fc1(x))
        #print(f"after fc1 and relu: {x.shape}")
        x = F.relu(self.fc2(x))
        #print(f"after fc2 and relu: {x.shape}")
        x = self.fc3(x)
        #print(f"after fc3: {x.shape}")
        return x

In [25]:
net = Net()

### Loss and optimizer

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)

### Train NN

In [27]:
# tensorboard summary writer
writer = SummaryWriter()

In [ ]:
running_loss = 0.0
for i, data in enumerate(test_dataloader, 0):
    # print(f"i: {i}, data: {data}")
    # break
    inputs, labels = data

    # zero parameter gradient #TODO: review optimizers
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    # write loss to runs for tensorboard
    try:
        writer.add_scalar("Loss", loss, i) #see https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html
    except(NameError):
        pass
    
    loss.backward()
    optimizer.step()

    # statisticshttps://pytohttps://pytorch.org/tutorials//beginner/onnx/export_simple_model_to_onnx_tutorial.htmlrch.org/tutorials//beginner/onnx/export_simple_model_to_onnx_tutorial.html
    # https://stackoverflow.com/questions/61092523/what-is-running-loss-in-pytorch-and-how-is-it-calculated
    running_loss += loss.item()
    if i%20 == 19:
        print(f"{i + 1:5d} loss: {running_loss / 20:.6f}")
        running_loss = 0.0

   20 loss: 2.227235
   40 loss: 2.231335
   60 loss: 2.205677
   80 loss: 2.235106
  100 loss: 2.203714
  120 loss: 2.231620
  140 loss: 2.211777
  160 loss: 2.224534
  180 loss: 2.214506
  200 loss: 2.221121
  220 loss: 2.217701
  240 loss: 2.212318
  260 loss: 2.225044


# visualize Net architecture

use [Netron](https://github.com/lutzroeder/netron) to visualize the net by exporting the net as an onnx file

In [ ]:
export_net = Net()
torch_input = torch.randn(64,80,40) #TODO: no idea of the dimensions are actually correct
# export as onnx
onnx_program = torch.onnx.dynamo_export(export_net, torch_input)
onnx_program.save("classifier_net.onnx")